# Process Concaténation model

In [ ]:
import torch
from Snt import Snt
from Matrice import Matrice
import Utils_data
import Utils_concat
import importlib
importlib.reload(Utils_data)
importlib.reload(Utils_concat)

id = 1850
for id in range(2308):
    print(f"[DEBUG] id: {id}")
    r_path=f"/home/getalp/lopezfab/lig/temp/temp/test_attn/{id}.json"
    data=Utils_data.lecture_data(r_path)
    _OUTPUT_PATH=f"/home/getalp/lopezfab/Documents/concat/{id}"

    # Traitement des Phrases extraites
    ssl = data['src_segments_labels']
    src = Snt(identifiant=-1, tokens=Utils_concat.ajoute_eos_tokens_src(_snt=data["src_tokens"].split(), src_segments_labels=ssl))
    src_cutted = Utils_concat.full_sentence_to_ctx_and_crt(src)
    for i in range(len(src_cutted)-1, -1, -1):
        if src_cutted[i] == ["<eos>"]:
            del src_cutted[i]
    if len(src_cutted) > 1:
        # Extraction des phrases de contexte et de la phrase courante
        ctxs = []
        for k in range(len(src_cutted[:-1])):
            ctxs.append(Snt(identifiant=int(data["id"]) - len(src_cutted[k:-1]), tokens = src_cutted[k]))
        full_ctx = Snt(identifiant= id - len(ctxs), tokens= ctxs[0].tokens)
        # print(f"[debug] len(full_ctx): {len(full_ctx)}")
        if len(ctxs) > 1:
            for snt in ctxs[1:]:
                full_ctx.tokens += snt.tokens
                # print(f"[debug] len(full_ctx): {len(full_ctx)}")
        crt = Snt(identifiant=data['id'], tokens= src_cutted[-1])
        full_ctx.tokens += crt.tokens

        # Extraction des différentes matrices à travers les 6 layers et les 8 têtes d'attention de chaque layer
        layers = []
        for layer in range(len(data['heads_enc_attn'])): # Pour chaque layer
            heads = []
            for head in range(len(data['heads_enc_attn'][layer])): # on extrait chaque tête par layer
                full_matrice = torch.tensor(data['heads_enc_attn'][layer][head])
                full_matrice = full_matrice.squeeze() # on supprime une dimension qui semble inutile (=1)
                heads.append(Matrice(full_matrice))
            layers.append(heads)

        # layers : L x [nb_heads x [torch.Tensor(N x N)]]
        # print((f"[DEBUG] taille des matrices: {layers[0][0].matrice.size()}"))
        # print(f"[DEBUG] taille des phrases complète vs tailles respectives: {sum([len(ctx) for ctx in ctxs] + [len(crt)])} vs. [{[len(ctx) for ctx in ctxs]}, {len(crt)}]")

        # Pour chaque layer, pour chaque tête d'attention, on découpe la matrice en combinaison de k3*k3, k3*k2... k2*k3, k2*k2,... crt*crt
        for layer in range(len(layers)):
            for head in range(len(layers[layer])):
                full_matrice_cutted = Utils_concat.cut_matrix_into_sentences(layers[layer][head], src_cutted)
                # Dernière liste correspond à la phrase courante vers les phrases de contexte et la phrase courante
                layers[layer][head].matrice = torch.cat([matrice.matrice for matrice in full_matrice_cutted[-1][:-1]], dim = 1) # Remplacer [:] par [:-1] pour n'avoir que le contexte
        # print(f"[DEBUG] taille de la matrice découpée: {[matrice.matrice.size() for matrice in full_matrice_cutted[-1]]}")
        # print(f"[DEBUG] taille de la matrice reconstituée: {layers[layer][head].matrice.size()}")
        # print(f"[DEBUG] élément matrice: {layers[0][0].matrice[0:2, 3:9]}")
        for layer in range(len(layers)):
            for head in range(len(layers[layer])):
                layers[layer][head].suppr_inf()
                layers[layer][head].norm_tensor()
                layers[layer][head].ecriture_xslx(crt=crt, ctx=full_ctx, precision= 4, absolute_folder=f"{_OUTPUT_PATH}/full_matrice/{layer}", filename=f"{head}", create_folder_path=True)

[DEBUG] id: 0
[DEBUG] id: 1
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/1/0
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/1/1
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/1/2
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/1/3
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/1/4
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/1/5
[DEBUG] id: 2
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/2/0
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/2/1
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/2/2
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/2/3
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/2/4
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat/2/5
[DEBUG] id: 3
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/concat

KeyboardInterrupt: 

In [11]:
import Utils_concat as uc
import importlib
importlib.reload(uc)

liste_10_x_10 = [[0 , 1, 2, 3, 4, 5, 6, 7, 8, 9], [10,11,12,13,14,15,16,17,18,19], [20,21,22,23,24,25,26,27,28,29], [30,31,32,33,34,35,36,37,38,39], [40,41,42,43,44,45,46,47,48,49], [50,51,52,53,54,55,56,57,58,59], [60,61,62,63,64,65,66,67,68,69], [70,71,72,73,74,75,76,77,78,79], [80,81,82,83,84,85,86,87,88,89], [90,91,92,93,94,95,96,97,98,99]]
snts = [Snt(identifiant=0, tokens = ["k3t1", "k3t2"]), Snt(identifiant=1, tokens = ["k2t1", "k2t2", "k2t3"]), Snt(identifiant=2, tokens = ["k1t1", "k1t2"]), Snt(identifiant=3, tokens = ["k0t1", "k0t2", "k0t3"])]
m1 = Matrice(torch.Tensor(liste_10_x_10))
print(uc.cut_matrix_into_sentences(m1, snts)[-1])


[{'_matrice': tensor([[70., 71.],
        [80., 81.],
        [90., 91.]])}, {'_matrice': tensor([[72., 73., 74.],
        [82., 83., 84.],
        [92., 93., 94.]])}, {'_matrice': tensor([[75., 76.],
        [85., 86.],
        [95., 96.]])}, {'_matrice': tensor([[77., 78., 79.],
        [87., 88., 89.],
        [97., 98., 99.]])}]


In [16]:
print(layers[0][0].matrice.size())
    
print(sum([len(ctx) for ctx in ctxs] + [len(crt)]))

torch.Size([138, 138])
138
